# Predicting Heart Disease

## Score: .95355

In [8]:
from pathlib import Path
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold, cross_val_predict
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
import xgboost as xgb
import lightgbm as lgb
import catboost as cb

DATA_DIR = Path("playground-series-s6e2")
OUTPUT_DIR = Path(".")
n_splits = 5
cv = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

In [9]:
train = pd.read_csv(DATA_DIR / "train.csv")
test = pd.read_csv(DATA_DIR / "test.csv")
print(f"Train: {train.shape}")
print(f"Test: {test.shape}")
train.head()

Train: (630000, 15)
Test: (270000, 14)


,id,Age,Sex,Chest pain type,BP,Cholesterol,FBS over 120,EKG results,Max HR,Exercise angina,ST depression,Slope of ST,Number of vessels fluro,Thallium,Heart Disease
0,0,58,1,4,152,239,0,0,158,1,3.6,2,2,7,Presence
1,1,52,1,1,125,325,0,2,171,0,0.0,1,0,3,Absence
2,2,56,0,2,160,188,0,2,151,0,0.0,1,0,3,Absence
3,3,44,0,3,134,229,0,2,150,0,1.0,2,0,3,Absence
4,4,58,1,4,140,234,0,2,125,1,3.8,2,3,3,Presence


In [10]:
target_col = "Heart Disease"
id_col = "id"
feature_cols = [c for c in train.columns if c not in (id_col, target_col)]

le = LabelEncoder()
y = le.fit_transform(train[target_col])

X_train = train[feature_cols].copy()
X_test = test[feature_cols].copy()

for col in feature_cols:
    if X_train[col].isna().any() or X_test[col].isna().any():
        med = X_train[col].median()
        X_train[col] = X_train[col].fillna(med)
        X_test[col] = X_test[col].fillna(med)

print(f"Features: {feature_cols}")
print(f"Target distribution: {pd.Series(y).value_counts().to_dict()}")

Features: ['Age', 'Sex', 'Chest pain type', 'BP', 'Cholesterol', 'FBS over 120', 'EKG results', 'Max HR', 'Exercise angina', 'ST depression', 'Slope of ST', 'Number of vessels fluro', 'Thallium']
Target distribution: {0: 347546, 1: 282454}


In [11]:
param_grid = {
    "max_depth": [4, 5, 6],
    "learning_rate": [0.03, 0.05],
    "min_child_weight": [2, 3, 5],
}
best_auc = 0
best_params = None
for max_depth in param_grid["max_depth"]:
    for learning_rate in param_grid["learning_rate"]:
        for min_child_weight in param_grid["min_child_weight"]:
            model = xgb.XGBClassifier(
                n_estimators=1000,
                max_depth=max_depth,
                min_child_weight=min_child_weight,
                learning_rate=learning_rate,
                subsample=0.7,
                colsample_bytree=0.7,
                random_state=42,
                eval_metric="auc",
            )
            oof = cross_val_predict(model, X_train, y, cv=cv, method="predict_proba")[:, 1]
            auc = roc_auc_score(y, oof)
            if auc > best_auc:
                best_auc = auc
                best_params = {"max_depth": max_depth, "learning_rate": learning_rate, "min_child_weight": min_child_weight}
print(f"Best XGB CV AUC: {best_auc:.5f}")
print(f"Best params: {best_params}")

Best XGB CV AUC: 0.95542
Best params: {'max_depth': 4, 'learning_rate': 0.05, 'min_child_weight': 5}


In [12]:
oof_xgb = cross_val_predict(
    xgb.XGBClassifier(n_estimators=1000, **best_params, subsample=0.7, colsample_bytree=0.7, random_state=42, eval_metric="auc"),
    X_train, y, cv=cv, method="predict_proba",
)[:, 1]
oof_lgb = cross_val_predict(
    lgb.LGBMClassifier(n_estimators=1000, max_depth=best_params["max_depth"], min_child_samples=20, learning_rate=best_params["learning_rate"], subsample=0.7, colsample_bytree=0.7, random_state=42, verbosity=-1),
    X_train, y, cv=cv, method="predict_proba",
)[:, 1]
oof_cb = cross_val_predict(
    cb.CatBoostClassifier(iterations=1000, depth=best_params["max_depth"], min_data_in_leaf=20, learning_rate=best_params["learning_rate"], subsample=0.7, colsample_bylevel=0.7, random_seed=42, verbose=0),
    X_train, y, cv=cv, method="predict_proba",
)[:, 1]

meta_X = np.column_stack([oof_xgb, oof_lgb, oof_cb])
meta = LogisticRegression(C=0.1, max_iter=500, random_state=42)
meta.fit(meta_X, y)
stack_auc = roc_auc_score(y, meta.predict_proba(meta_X)[:, 1])
print(f"Stack (meta on OOF) CV AUC: {stack_auc:.5f}")

Stack (meta on OOF) CV AUC: 0.95544


In [13]:
xgb_list = []
for seed in [42, 43, 44, 45, 46]:
    m = xgb.XGBClassifier(n_estimators=1000, **best_params, subsample=0.7, colsample_bytree=0.7, random_state=seed, eval_metric="auc")
    m.fit(X_train, y)
    xgb_list.append(m.predict_proba(X_test)[:, 1])
lgb_list = []
for seed in [42, 43, 44]:
    m = lgb.LGBMClassifier(n_estimators=1000, max_depth=best_params["max_depth"], min_child_samples=20, learning_rate=best_params["learning_rate"], subsample=0.7, colsample_bytree=0.7, random_state=seed, verbosity=-1)
    m.fit(X_train, y)
    lgb_list.append(m.predict_proba(X_test)[:, 1])
cb_list = []
for seed in [42, 43, 44]:
    m = cb.CatBoostClassifier(iterations=1000, depth=best_params["max_depth"], min_data_in_leaf=20, learning_rate=best_params["learning_rate"], subsample=0.7, colsample_bylevel=0.7, random_seed=seed, verbose=0)
    m.fit(X_train, y)
    cb_list.append(m.predict_proba(X_test)[:, 1])
test_meta_X = np.column_stack([np.mean(xgb_list, axis=0), np.mean(lgb_list, axis=0), np.mean(cb_list, axis=0)])
test_proba = meta.predict_proba(test_meta_X)[:, 1]

In [14]:
sub = pd.DataFrame({"id": test["id"], "Heart Disease": test_proba})
sub.to_csv(OUTPUT_DIR / "submission.csv", index=False)
print(f"Submission saved to {OUTPUT_DIR / 'submission.csv'}")
sub.head(10)

Submission saved to submission.csv


,id,Heart Disease
0,630000,0.948485
1,630001,0.039354
2,630002,0.959882
3,630003,0.038595
4,630004,0.123816
5,630005,0.959247
6,630006,0.038778
7,630007,0.733155
8,630008,0.961117
9,630009,0.040753
